In [ ]:
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import warnings
import numpy as np
import pandas as pd
from IPython.display import display
import copy

import sys
sys.path.append('../Data')

In [ ]:
# Load Data

X = pd.read_csv('../Data/Benzene Training Data/[Benz] ANM_X.csv')
X_lexi = pd.read_csv('../Data/Benzene Training Data/[Benz] ANM_X_lexi.csv')
X_lexi_nd = pd.read_csv('../Data/Benzene Training Data/[Benz] ANM_X_lexi_nd.csv')
X_sorted = pd.read_csv('../Data/Benzene Training Data/[Benz] ANM_X_sorted.csv')
X_coulomb = pd.read_csv('../Data/Benzene Training Data/[Benz] ANM_X_coulomb.csv')

inv_dist_X = pd.read_csv('../Data/Benzene Training Data/[Benz] inv_dist_X.csv')
inv_dist_X_lexi = pd.read_csv('../Data/Benzene Training Data/[Benz] inv_dist_X_lexi.csv')
inv_dist_X_lexi_nd = pd.read_csv('../Data/Benzene Training Data/[Benz] inv_dist_X_lexi_nd.csv')
inv_dist_X_sorted = pd.read_csv('../Data/Benzene Training Data/[Benz] inv_dist_X_sorted.csv')
inv_dist_X_coulomb = pd.read_csv('../Data/Benzene Training Data/[Benz] inv_dist_X_coulomb.csv')

rand_X = pd.read_csv('../Data/Benzene Training Data/[Benz] rand_X.csv')
rand_X_lexi = pd.read_csv('../Data/Benzene Training Data/[Benz] rand_X_lexi.csv')
rand_X_lexi_nd = pd.read_csv('../Data/Benzene Training Data/[Benz] rand_X_lexi_nd.csv')
rand_X_sorted = pd.read_csv('../Data/Benzene Training Data/[Benz] rand_X_sorted.csv')
rand_X_coulomb = pd.read_csv('../Data/Benzene Training Data/[Benz] rand_X_coulomb.csv')

y_energy = pd.read_csv("../Data/Benzene Training Data/[Benz] y_energy.csv")
y_elec = pd.read_csv("../Data/Benzene Training Data/[Benz] y_elec.csv")
y_delta_elec = pd.read_csv("../Data/Benzene Training Data/[Benz] y_delta_elec.csv")
y_delta_energy = pd.read_csv("../Data/Benzene Training Data/[Benz] y_delta_energy.csv")